In [19]:
import sys
sys.path.append('../deepsurv')
import deepsurv

#from deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lasagne
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [24]:
eln_clin_demo_comp = [0]+list(range(153,180))
eln_cyto_gen_comp = list(range(171))
eln_cyto_comp = [0] + list(range(84,171))
eln_gen_comp = list(range(84)) + list(range(153,171))
clin_demo_comp = list(range(153,180))
cyto_gen_comp = list(range(1,171))
cyto_comp = list(range(84,171))
gen_comp = list(range(1,84))+list(range(153,171))
clin_demo_cyto_gen_comp = list(range(1,180))
comp = list(range(153,171))
clin_demo_cyto_gen = list(range(1,153))

In [28]:
train_df = pd.read_table('df_prognosis_features_ready_final_component.tsv').iloc[:,clin_demo_cyto_gen+[180,181]]
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)

In [29]:
def dataframe_to_deepsurv_ds(df, event_col = 'os_status', time_col = 'os'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    #x=x_df
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

In [30]:
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [86]:
train_df = pd.read_table('df_prognosis_features_ready_final_component.tsv').iloc[:,cyto_gen_comp+[180,181]]
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')
hid_layers=[100,100,10]
#network=[None]*2
#metrics=[None]*2
for i in range(1):

    hyperparams = {
        'L2_reg': 1.0,
        'batch_norm': True,
        'dropout': 0.4,
        'hidden_layers_sizes': [100,100],
        'learning_rate': 1e-3,
        'lr_decay': 0.001,
        'momentum': 0.9,
        'n_in': train_data['x'].shape[1],
        'activation':'rectify'
        #'standardize': True
    }
    np.random.seed(17)
    #network[i] = deepsurv.DeepSurv(**hyperparams)
    #network[i].restored_update_params = None 
    logger = deepsurv.deepsurv_logger.TensorboardLogger(name = 'DeepSurvLogger', logdir = './logs/tensorboard/')
    np.random.seed(17)
    #update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. 
    #update_fn=lasagne.updates.rmsprop
    n_epochs = 5000
    val_frequency=250
    patience=10000 # minimum number of epochs to train for
    metrics[i] =  network[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)
    print (metrics[i]['valid_c-index'])

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000    |                         | - loss: 6.3432 - ci: 0.7797
2019-06-04 16:43:37,001 - Training step 0/5000

In [87]:
metrics[0]['valid_c-index']

[(0, 0.6512132472131474),
 (250, 0.6503653457692212),
 (500, 0.6505648519913215),
 (750, 0.6500660864360708),
 (1000, 0.6499164567694955),
 (1250, 0.6498167036584453),
 (1500, 0.6493179381031946),
 (1750, 0.649243123269907),
 (2000, 0.6488441108257064),
 (2250, 0.6488939873812315),
 (2500, 0.6486196663258436),
 (2750, 0.6481957156038805),
 (3000, 0.64799620938178),
 (3250, 0.6474475672710043),
 (3500, 0.6468739868824659),
 (3750, 0.6464749744382653),
 (4000, 0.6459013940497269),
 (4250, 0.6454774433277638),
 (4500, 0.6449537394947505),
 (4750, 0.6448539863837004)]

In [88]:
for i in range(1):
    print (network[i].get_concordance_index(x=val_data['x'],t=val_data['t'],e=val_data['e']),network[i].get_concordance_index(x=test_data['x'],t=test_data['t'],e=test_data['e']),network[i].hyperparams)

0.6442305294396369 0.663980091514811 {'n_in': 170, 'learning_rate': 1e-05, 'hidden_layers_sizes': [100, 100], 'lr_decay': 0.001, 'momentum': 0.9, 'L2_reg': 1.0, 'L1_reg': 0.0, 'activation': 'rectify', 'dropout': 0.4, 'batch_norm': True, 'standardize': False}


In [ ]:
metrics[i] =  network[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)

In [ ]:
train_df = pd.read_table('df_prognosis_features_ready_final_component.tsv').iloc[:,clin_demo_cyto_gen_comp+[180,181]]
train,test = train_test_split(train_df,test_size=0.2,random_state=17)
train,val = train_test_split(train,test_size=0.2,random_state=17)
### Min Max Scaler Normalization and good format of the data to feed the MLP network
scale = preprocessing.MinMaxScaler().fit(train)
df_train = pd.DataFrame(scale.transform(train.values), columns=train.columns, index=train.index)
df_val = pd.DataFrame(scale.transform(val.values), columns=val.columns, index=val.index)
df_test = pd.DataFrame(scale.transform(test.values), columns=test.columns, index=test.index)
train_data = dataframe_to_deepsurv_ds(df_train, event_col = 'os_status', time_col= 'os')
val_data = dataframe_to_deepsurv_ds(df_val, event_col = 'os_status', time_col= 'os')
test_data = dataframe_to_deepsurv_ds(df_test, event_col = 'os_status', time_col= 'os')
hid_layers=[[100,100,10],[200,500,10],[250,100,100],[500,100,100]]
drop=[0.2,0.2,0.2,0.2]
network1=[None]*4
metrics1=[None]*4
for i in range(4):

    hyperparams = {
        'L2_reg': 10.0,
        'batch_norm': True,
        'dropout': 0.4,
        'hidden_layers_sizes': hid_layers[i],
        'learning_rate': 1e-05,
        'lr_decay': 0.001,
        'momentum': 0.9,
        'n_in': train_data['x'].shape[1],
        'activation':'rectify'
        #'standardize': True
    }
    np.random.seed(17)
    network1[i] = deepsurv.DeepSurv(**hyperparams)
    network1[i].restored_update_params = None 
    logger = deepsurv.deepsurv_logger.TensorboardLogger(name = 'DeepSurvLogger', logdir = './logs/tensorboard/')
    np.random.seed(17)
    #update_fn=lasagne.updates.nesterov_momentum # The type of optimizer to use. 
    update_fn=lasagne.updates.rmsprop
    n_epochs = 12000
    val_frequency=250
    patience=10000 # minimum number of epochs to train for
    metrics1[i] =  network1[i].train(train_data,val_data, n_epochs=n_epochs, logger=logger, update_fn=update_fn,patience=patience,validation_frequency=val_frequency)